In [2]:
from particletracking import dataframes, statistics
import numpy as np
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
with dataframes.DataStore('/media/data/Data/FirstOrder/PhaseDiagram/DimpledPlateFeb2021/1900.hdf5') as data:
    df = data.df

In [4]:
points = df.loc[30000]

In [5]:
points.x.max()

2183

In [6]:
from scipy import ndimage, signal

In [7]:
def gkern(kernlen=21, std=3):
    gkern1d = signal.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d

In [8]:
std = points.r.mean()/10

In [9]:
r = int(points.r.mean())
r

17

In [10]:
def create_view(points, std):
    r = int(points.r.mean())
    std = r * std
    k = gkern(2*r, std)
    grid = np.zeros((points.x.max(), points.y.max()))
    for i, p in points.iterrows():
        
        x = p.x
        y = p.y
        try:
            grid[x-r:x+r, y-r:y+r] += k
        except ValueError as e:
            pass
    return grid


In [11]:
grid = create_view(points, 0.1)

In [12]:
plt.imshow(grid)

In [13]:
a = np.array(((1, 1, 1), (2, 2, 2), (3, 3, 3)))
b = np.ones((100, 100))

In [14]:
transform = np.log(np.abs(np.fft.fft2(grid)))

In [15]:
transform_shifted = np.fft.fftshift(transform)

In [16]:
shape = transform.shape
top_left = transform[:shape[0]//2, :shape[1]//2]

In [17]:
plt.imshow(top_left)

In [18]:
from skimage.feature import peak_local_max
from skimage import data, img_as_float

In [21]:
peaks = peak_local_max(transform_shifted, min_distance=r)

In [39]:
plt.imshow(transform_shifted)
plt.plot(peaks[14, 1], peaks[14, 0], '.')

In [28]:
central_peak = peaks[0]
central_peak

array([1091,  969])

In [29]:
from scipy import spatial

In [30]:
tree = spatial.KDTree(peaks[1:])

In [51]:
dist, closest_index = tree.query(central_peak, 12)
closest_index += 1

In [52]:
ring_peak = peaks[closest_index, :]

In [59]:
chosen_peak = ring_peak[8, :] # The peaks of interest should be between indices 6 and 11 so choose 8 to increase selection likelihood

In [98]:
plt.imshow(transform_shifted)
plt.plot(chosen_peak[1], chosen_peak[0], '.')

In [62]:
# Get the reciprocal lattice vector - wrong units still though
vector = chosen_peak - central_peak
vector

array([-32,  48])

In [63]:
spacing = 1 / (len(grid)) # 

In [65]:
vector = vector * spacing
vector

array([-0.01465873,  0.02198809])

In [73]:
points[['x', 'y']].values.shape

(1994, 2)

In [74]:
torder = np.exp(1j*np.dot(points[['x', 'y']].values, vector))

In [81]:
torder_phase = np.arctan2(np.imag(torder),np.real(torder))

In [82]:
plt.scatter(points.x, points.y, c=torder_phase)

In [78]:
plt.hist(np.real(torder))

(array([398., 181., 163., 132., 113., 137., 146., 136., 179., 409.]),
 array([-9.99997240e-01, -7.99997700e-01, -5.99998161e-01, -3.99998621e-01,
        -1.99999082e-01,  4.57579569e-07,  1.99999997e-01,  3.99999537e-01,
         5.99999076e-01,  7.99998615e-01,  9.99998155e-01]),
 <BarContainer object of 10 artists>)

In [87]:
def transform_view(view):
    transform = np.fft.fft2(view)
    transform = np.abs(transform)
    transform = np.log(transform) # Put the numbers on a nicer scale for viewing
    transform = np.fft.fftshift(transform)
    return transform

In [103]:
def get_peaks(view):
    peaks = peak_local_max(view, 20)
    tree = spatial.KDTree(peaks[1:])
    dist, indices = tree.query(peaks[0], 12)
    chosen_peak = peaks[indices[8]] # Indices 6 to 11 represent peaks of interest
    return peaks[0], chosen_peak

In [112]:
def get_vector_from_peaks(central, ring, spacing):
    vector = ring - central
    vector = vector * spacing
    return vector    

In [143]:
def primary_reciprocal_lattice_vector(points):
    points *= 10
    particle_view = create_view(points, 0.1)
    particle_view_transformed = transform_view(particle_view)
    central_peak, ring_peak = get_peaks(particle_view_transformed)
    spacing = 1/len(particle_view)
    vector = get_vector_from_peaks(central_peak, ring_peak, spacing)
    return vector
    

In [144]:
G = primary_reciprocal_lattice_vector(points[['x', 'y', 'r']])

/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [126]:
def get_torder(points, G):
    return np.exp(1j*np.dot(points[['x', 'y']].values, G))

In [127]:
torder = get_torder(points, G)

In [147]:
torder**2

array([-0.9579568 -0.28691248j, -0.16424443-0.98641967j,
        0.86718679+0.497983j  , ..., -0.92650457+0.37628352j,
       -0.91785506+0.39691572j, -0.32095743+0.94709362j])

In [134]:
susceptibility = np.mean(np.abs(torder**2)) - np.mean(np.abs(torder))**2
susceptibility

0.0

In [136]:
def get_susceptibility(param):
    a = np.mean(np.abs(param**2))
    b = np.mean(np.abs(param))**2
    print(a, b)
    return a - b

In [137]:
s = get_susceptibility(torder)
s

1.0 1.0


0.0

This isn't the susceptibility. Need to do a time average.

Also Sun calculated the susceptibility for multiple sub-boxes of different sizes and extrapolated to find the susceptiblity at the infinite size limit.